In [ ]:
!wget https://raw.githubusercontent.com/alimbekovKZ/ods_homework/master/data/wiki_machine_learning.csv

--2020-11-09 10:04:25--  https://raw.githubusercontent.com/alimbekovKZ/ods_homework/master/data/wiki_machine_learning.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31043 (30K) [text/plain]
Saving to: ‘wiki_machine_learning.csv’

wiki_machine_learni 100%[===================>]  30.32K  --.-KB/s    in 0.01s   

2020-11-09 10:04:26 (2.21 MB/s) - ‘wiki_machine_learning.csv’ saved [31043/31043]



In [ ]:
!pip install tsfresh

     |████████████████████████████████| 92kB 2.2MB/s 
     |████████████████████████████████| 665kB 8.6MB/s 
     |████████████████████████████████| 92kB 5.7MB/s 
     |████████████████████████████████| 102kB 7.3MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=63bf61bb1d49674fa226ec4ba9a93c8c937165c555f3e2eac97fa8e23e6a4067
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=a6dcd58ce7fa5d6464d519becf064a63f8b3202e5ed7ee75eb28e148dd6c427b
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built contextvars locket
ERROR: tensorflow-probability 0.11.0 has requirement cloudpickle==1.3, but you'll have cloudpickle 1.6.0 which is incompatible.
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Suc

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from tsfresh import extract_relevant_features

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('wiki_machine_learning.csv', sep = ' ')
df = df[df['count'] != 0]
df.head()

,date,count,lang,page,rank,month,title
81,2015-01-01,1414,en,Machine_learning,8708,201501,Machine_learning
80,2015-01-02,1920,en,Machine_learning,8708,201501,Machine_learning
79,2015-01-03,1338,en,Machine_learning,8708,201501,Machine_learning
78,2015-01-04,1404,en,Machine_learning,8708,201501,Machine_learning
77,2015-01-05,2264,en,Machine_learning,8708,201501,Machine_learning


In [ ]:
df.date = pd.to_datetime(df.date)

In [ ]:
predictions = 30

df = df[['date', 'count']]
df.columns = ['ds', 'y']
train_df = df[:-predictions].copy()

In [ ]:
from tsfresh import extract_features
extracted_features = extract_features(df.reset_index(), column_id="index", column_sort="ds")

Feature Extraction: 100%|██████████| 5/5 [00:11<00:00,  2.36s/it]


In [ ]:
extracted_features.shape

(383, 779)

In [ ]:
extracted_features

,y__variance_larger_than_standard_deviation,y__has_duplicate_max,y__has_duplicate_min,y__has_duplicate,y__sum_values,y__abs_energy,y__mean_abs_change,y__mean_change,y__mean_second_derivative_central,y__median,y__mean,y__length,y__standard_deviation,y__variation_coefficient,y__variance,y__skewness,y__kurtosis,y__absolute_sum_of_changes,y__longest_strike_below_mean,y__longest_strike_above_mean,y__count_above_mean,y__count_below_mean,y__last_location_of_maximum,y__first_location_of_maximum,y__last_location_of_minimum,y__first_location_of_minimum,y__percentage_of_reoccurring_values_to_all_values,y__percentage_of_reoccurring_datapoints_to_all_datapoints,y__sum_of_reoccurring_values,y__sum_of_reoccurring_data_points,y__ratio_value_number_to_time_series_length,y__sample_entropy,y__maximum,y__minimum,y__benford_correlation,y__time_reversal_asymmetry_statistic__lag_1,y__time_reversal_asymmetry_statistic__lag_2,y__time_reversal_asymmetry_statistic__lag_3,y__c3__lag_1,y__c3__lag_2,...,y__number_crossing_m__m_0,y__number_crossing_m__m_-1,y__number_crossing_m__m_1,y__energy_ratio_by_chunks__num_segments_10__segment_focus_0,y__energy_ratio_by_chunks__num_segments_10__segment_focus_1,y__energy_ratio_by_chunks__num_segments_10__segment_focus_2,y__energy_ratio_by_chunks__num_segments_10__segment_focus_3,y__energy_ratio_by_chunks__num_segments_10__segment_focus_4,y__energy_ratio_by_chunks__num_segments_10__segment_focus_5,y__energy_ratio_by_chunks__num_segments_10__segment_focus_6,y__energy_ratio_by_chunks__num_segments_10__segment_focus_7,y__energy_ratio_by_chunks__num_segments_10__segment_focus_8,y__energy_ratio_by_chunks__num_segments_10__segment_focus_9,y__ratio_beyond_r_sigma__r_0.5,y__ratio_beyond_r_sigma__r_1,y__ratio_beyond_r_sigma__r_1.5,y__ratio_beyond_r_sigma__r_2,y__ratio_beyond_r_sigma__r_2.5,y__ratio_beyond_r_sigma__r_3,y__ratio_beyond_r_sigma__r_5,y__ratio_beyond_r_sigma__r_6,y__ratio_beyond_r_sigma__r_7,y__ratio_beyond_r_sigma__r_10,y__count_above__t_0,y__count_below__t_0,y__lempel_ziv_complexity__bins_2,y__lempel_ziv_complexity__bins_3,y__lempel_ziv_complexity__bins_5,y__lempel_ziv_complexity__bins_10,y__lempel_ziv_complexity__bins_100,y__fourier_entropy__bins_2,y__fourier_entropy__bins_3,y__fourier_entropy__bins_5,y__fourier_entropy__bins_10,y__fourier_entropy__bins_100,y__permutation_entropy__dimension_3__tau_1,y__permutation_entropy__dimension_4__tau_1,y__permutation_entropy__dimension_5__tau_1,y__permutation_entropy__dimension_6__tau_1,y__permutation_entropy__dimension_7__tau_1
1,0.0,0.0,0.0,0.0,2703.0,7306209.0,NaN,NaN,NaN,2703.0,2703.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,2703.0,2703.0,0.295657,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,1783.0,3179089.0,NaN,NaN,NaN,1783.0,1783.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1783.0,1783.0,0.864123,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,2895.0,8381025.0,NaN,NaN,NaN,2895.0,2895.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,2895.0,2895.0,0.295657,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,0.0,2985.0,8910225.0,NaN,NaN,NaN,2985.0,2985.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,2985.0,2985.0,0.295657,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0.0,0.0,0.0,2932.0,8596624.0,NaN,NaN,NaN,2932.0,2932.0,1.0,0

In [ ]:
type(extracted_features)

pandas.core.frame.DataFrame

In [ ]:
train_df = extracted_features[:-predictions].copy()

In [ ]:
train_df = train_df.dropna(axis=1, how='all')

In [ ]:
pipeline = make_pipeline(MinMaxScaler(), RandomForestRegressor(random_state=42))
pipeline.fit(train_df, df.y[:-predictions])

Pipeline(memory=None,
         steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('randomforestregressor',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=None,
                                       max_features='auto', max_leaf_nodes=None,
                                       max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=100, n_jobs=None,
                                       oob_score=False, random_state=42,
                                       verbose=0, warm_start=False))],
         verbose=False)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
y_test = extracted_features[353:].dropna(axis=1, how='all')

In [ ]:
preds_pipeline = pipeline.predict(y_test)

print(mean_absolute_percentage_error(df[353:].y, preds_pipeline))
print(mean_absolute_percentage_error(sum(df[353:].y), sum(preds_pipeline)))

31.07428354453911
4.799520002041403
